<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignmnet 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [83]:
''' Imports '''
#!pip install tensorflow==1.14.0 keras==2.2.4
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras import backend

''' Importing the DataSet '''
from keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((404, 13), (404,), (102, 13), (102,))

In [84]:
''' Building the Model '''
model = Sequential()
model.add(Dense(13, input_dim=13, activation='relu'))
model.add(BatchNormalization()) # normalizing the data 
model.add(Dense(13, activation='relu'))
model.add(Dense(1, activation='relu'))

''' Compiling the model '''
model.compile(loss='mean_absolute_percentage_error',
              optimizer='adam',
              metrics=['accuracy'])

In [85]:
''' Fitting the model '''
fitting = model.fit(x_train, y_train, epochs=150)
scores = model.evaluate(x_test, y_test)
print((scores[1]*100))

Epoch 1/150
404/404 [==============================] - 1s 3ms/step - loss: 99.2510 - acc: 0.0000e+00
Epoch 2/150
404/404 [==============================] - 0s 69us/step - loss: 98.2096 - acc: 0.0000e+00
Epoch 3/150
404/404 [==============================] - 0s 68us/step - loss: 96.7795 - acc: 0.0000e+00
Epoch 4/150
404/404 [==============================] - 0s 70us/step - loss: 94.9901 - acc: 0.0000e+00
Epoch 5/150
404/404 [==============================] - 0s 69us/step - loss: 92.8047 - acc: 0.0000e+00
Epoch 6/150
404/404 [==============================] - 0s 61us/step - loss: 90.2964 - acc: 0.0000e+00
Epoch 7/150
404/404 [==============================] - 0s 70us/step - loss: 87.6135 - acc: 0.0025
Epoch 8/150
404/404 [==============================] - 0s 69us/step - loss: 84.8956 - acc: 0.0000e+00
Epoch 9/150
404/404 [==============================] - 0s 71us/step - loss: 81.6961 - acc: 0.0000e+00
Epoch 10/150
404/404 [==============================] - 0s 64us/step - loss: 78.5864 - 

404/404 [==============================] - 0s 58us/step - loss: 16.2928 - acc: 0.0124
Epoch 84/150
404/404 [==============================] - 0s 61us/step - loss: 16.7143 - acc: 0.0025
Epoch 85/150
404/404 [==============================] - 0s 60us/step - loss: 16.1721 - acc: 0.0124
Epoch 86/150
404/404 [==============================] - 0s 57us/step - loss: 16.2404 - acc: 0.0173
Epoch 87/150
404/404 [==============================] - 0s 52us/step - loss: 16.6612 - acc: 0.0099
Epoch 88/150
404/404 [==============================] - 0s 57us/step - loss: 15.8853 - acc: 0.0198
Epoch 89/150
404/404 [==============================] - 0s 56us/step - loss: 16.2696 - acc: 0.0074
Epoch 90/150
404/404 [==============================] - 0s 58us/step - loss: 16.1882 - acc: 0.0099
Epoch 91/150
404/404 [==============================] - 0s 50us/step - loss: 16.1625 - acc: 0.0124
Epoch 92/150
404/404 [==============================] - 0s 59us/step - loss: 16.2647 - acc: 0.0124
Epoch 93/150
404/404 [=

In [86]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

y_pred = linear_model.predict(x_test)

print('Mean Square error', mean_squared_error(y_test, y_pred),
      'Mean Absolute Error', mean_absolute_error(y_test, y_pred))

# ''' Niceeee so 150 epochs of a total vanilla neural network did better than linear regression '''

Mean Square error 23.195599256422938 Mean Absolute Error 3.4641858124067126


In [87]:
''' Now to better optimize the neural net and try to get a very very low mean aboslute error 
    Along with some feature engineering
'''

' Now to better optimize the neural net and try to get a very very low mean aboslute error \n    Along with some feature engineering\n'

In [88]:
model_2 = Sequential()
model_2.add(Dense(169, input_dim=13, activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dense(13, activation='sigmoid'))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(loss='mean_absolute_percentage_error',
              optimizer='adam',
              metrics=['accuracy'])

In [89]:
fitting = model.fit(x_train, y_train, epochs=150)
scores = model.evaluate(x_test, y_test)

Epoch 1/150
404/404 [==============================] - 1s 2ms/step - loss: 15.1227 - acc: 0.0223
Epoch 2/150
404/404 [==============================] - 0s 69us/step - loss: 14.7439 - acc: 0.0124
Epoch 3/150
404/404 [==============================] - 0s 65us/step - loss: 15.0046 - acc: 0.0074
Epoch 4/150
404/404 [==============================] - 0s 67us/step - loss: 14.7474 - acc: 0.0124
Epoch 5/150
404/404 [==============================] - 0s 69us/step - loss: 15.3418 - acc: 0.0149
Epoch 6/150
404/404 [==============================] - 0s 65us/step - loss: 15.6506 - acc: 0.0025
Epoch 7/150
404/404 [==============================] - 0s 75us/step - loss: 15.1647 - acc: 0.0099
Epoch 8/150
404/404 [==============================] - 0s 81us/step - loss: 14.7330 - acc: 0.0149
Epoch 9/150
404/404 [==============================] - 0s 79us/step - loss: 14.7179 - acc: 0.0099
Epoch 10/150
404/404 [==============================] - 0s 68us/step - loss: 15.1063 - acc: 0.0025
Epoch 11/150
404/404

404/404 [==============================] - 0s 65us/step - loss: 14.6794 - acc: 0.0099
Epoch 84/150
404/404 [==============================] - 0s 66us/step - loss: 14.2316 - acc: 0.0198
Epoch 85/150
404/404 [==============================] - 0s 64us/step - loss: 13.9103 - acc: 0.0124
Epoch 86/150
404/404 [==============================] - 0s 61us/step - loss: 14.3297 - acc: 0.0074
Epoch 87/150
404/404 [==============================] - 0s 65us/step - loss: 14.8176 - acc: 0.0124
Epoch 88/150
404/404 [==============================] - 0s 54us/step - loss: 14.3539 - acc: 0.0198
Epoch 89/150
404/404 [==============================] - 0s 59us/step - loss: 14.2930 - acc: 0.0149
Epoch 90/150
404/404 [==============================] - 0s 56us/step - loss: 13.7132 - acc: 0.0099
Epoch 91/150
404/404 [==============================] - 0s 86us/step - loss: 14.3090 - acc: 0.0124
Epoch 92/150
404/404 [==============================] - 0s 75us/step - loss: 14.5626 - acc: 0.0099
Epoch 93/150
404/404 [=

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [63]:
from keras.datasets import fashion_mnist 
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [64]:
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0,   0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,
          1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          0,  36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,
          0,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,
          0, 102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,
         10,   0],
       [  

In [65]:
''' Preprocessing the data '''

X_train = x_train.reshape(60000, 784)
X_test = x_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], X_test.shape[0])

60000 10000


In [66]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [67]:
''' Further Preprocessing '''
y_train = keras.utils.to_categorical(y_train, 10) # 10 for the number of classes
y_test = keras.utils.to_categorical(y_test, 10) # 10 for the number of classes 

In [68]:
''' Making the Model '''
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=784))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [70]:
''' Compiling the model '''
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics = ['accuracy'])
history = model.fit(X_train, y_train,
                   batch_size = 128,
                   epochs = 15,
                   verbose=1,
                   validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 115us/step - loss: 0.3897 - acc: 0.8578 - val_loss: 0.4018 - val_acc: 0.8595
Epoch 2/10
60000/60000 [==============================] - 6s 107us/step - loss: 0.3580 - acc: 0.8702 - val_loss: 0.3977 - val_acc: 0.8614
Epoch 3/10
60000/60000 [==============================] - 6s 108us/step - loss: 0.3414 - acc: 0.8776 - val_loss: 0.3708 - val_acc: 0.8663
Epoch 4/10
60000/60000 [==============================] - 6s 105us/step - loss: 0.3289 - acc: 0.8818 - val_loss: 0.3553 - val_acc: 0.8796
Epoch 5/10
60000/60000 [==============================] - 6s 103us/step - loss: 0.3209 - acc: 0.8855 - val_loss: 0.3948 - val_acc: 0.8584
Epoch 6/10
60000/60000 [==============================] - 6s 97us/step - loss: 0.3125 - acc: 0.8876 - val_loss: 0.3694 - val_acc: 0.8709
Epoch 7/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.3101 - acc: 0.8889 - val_loss: 0.3597 - v

In [71]:
''' Evaluating the model '''
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

Test Loss:  0.3420857192158699
Test Accuracy:  0.8869


## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.